# MNIST Digit Recognizer

**Authors: Clement, Calvin, Tilova**

---

Welcome to the very first project of the **Tequila Chicas**! We will be classifying images of hand written numbers to their corresponding digits. This project follows the guidelines and uses the data set provide from the Kaggle Competition [here](https://www.kaggle.com/competitions/digit-recognizer/overview). 

## Introduction  

**MARKDOWN**

<a id = 'toc'></a>
    
## Table of Contents
---
1. [Convolutional Neural Network](#CNN)

**Importing Libraries**

In [6]:
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Train_Test_Split
from sklearn.model_selection import train_test_split

# Scaling
from sklearn.preprocessing import StandardScaler

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# ignores the filter warnings
import warnings
warnings.filterwarnings('ignore')

<a id = 'CNN'></a>
### 1. Convolutional Neural Network
---
Loading the test and train set CSVs files.

In [14]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df_train.shape, df_test.shape

((42000, 785), (28000, 784))

We need to set our independent (X) and dependent (y) variables as `numpy arrays` from the dataset.

In [15]:
X = df_train.iloc[:, 1:].to_numpy()
y = df_train.iloc[:, 0].to_numpy()

# sanity check
print(X.shape, y.shape)

(42000, 784) (42000,)


We will perform a **train_test_split()** to split our dataset into train and validation sets.
- Validation size of 25% of the data.
- Stratify=y to make sure distribution of the classes remain the same in both training and validation set.

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, stratify=y)
X_train.shape, y_train.shape

((31500, 784), (31500,))

#### 1.1 Image Preprocessing

---

**Steps**
1. Scale the data
2. Reshape the 1-D array into 2-D
2. Convert 2-D array into Torch tensors

In [18]:
# instantiate standard scaler
ss = StandardScaler()

# fit and transform training
X_train = ss.fit_transform(X_train)

# ONLY transform X_val
X_val = ss.transform(X_val)

In [20]:
# reshape training & validation
X_train = np.array(X_train).reshape(-1, 28, 28)
X_val = np.array(X_val).reshape(-1, 28, 28)

# sanity check
print(X_train.shape, X_val.shape)

(31500, 28, 28) (10500, 28, 28)


In [21]:
### To torch tensors ###
# Independent Variables
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)

# Dependent Variable
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

# Sanity Check
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

torch.Size([31500, 28, 28]) torch.Size([31500]) torch.Size([10500, 28, 28]) torch.Size([10500])
